# <h1><center>MuJoCo物理引擎完整教程 - 中文深度解析版  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

本教程为[**MuJoCo**物理引擎](https://github.com/google-deepmind/mujoco#readme)提供全面的中文入门指导，深入讲解Python API的使用方法和物理仿真原理。

**教程特色：**
- 🎯 从零开始，循序渐进掌握MuJoCo核心概念
- 🔬 深入剖析物理仿真原理和实现细节
- 💡 丰富的实例演示和可视化效果
- 🚀 涵盖从基础到高级的完整知识体系

<!-- Copyright 2021 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

# 环境配置与依赖安装

In [ ]:
# 安装MuJoCo包
%pip install mujoco

# 设置GPU渲染（如果在Colab环境中运行，请取消下面的注释）
# from google.colab import files
import distutils.util
import os
import subprocess

# 检查是否有GPU可用（Colab环境专用）
# if subprocess.run('nvidia-smi').returncode:
#   raise RuntimeError(
#       'Cannot communicate with GPU. '
#       'Make sure you are using a GPU Colab runtime. '
#       'Go to the Runtime menu and select Choose runtime type.')

# 添加ICD配置以便glvnd可以找到Nvidia EGL驱动
# 这通常作为Nvidia驱动包的一部分安装，但Colab内核不通过APT安装驱动
# 因此ICD配置可能缺失
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
# NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
# if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
#   with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
#     f.write("""{
#     "file_format_version" : "1.0.0",
#     "ICD" : {
#         "library_path" : "libEGL_nvidia.so.0"
#     }
# }
# """)

# 配置MuJoCo使用EGL渲染后端（需要GPU）
# print('Setting environment variable to use GPU rendering:')
# %env MUJOCO_GL=egl

# 检查安装是否成功
try:
  print('检查MuJoCo安装是否成功...')
  import mujoco
  # 尝试创建一个空模型来验证安装
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      '安装过程中出现问题。请检查上面的输出信息。\n'
      '如果使用Colab运行时，请确保启用了GPU加速：\n'
      '进入"运行时"菜单，选择"更改运行时类型"。')

print('安装成功！')

# 导入其他必要的库和辅助函数
import time
import itertools
import numpy as np

# 图形和绘图相关
print('安装媒体处理库...')
# 检查并安装ffmpeg（用于视频处理）
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
# 安装mediapy（用于显示图像和视频）
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# 设置NumPy打印选项，使输出更易读
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# 清除输出（使notebook更整洁）
from IPython.display import clear_output
clear_output()

# 第一部分：MuJoCo基础概念与模型加载

让我们从创建一个简单的模型开始，逐步理解MuJoCo的核心概念：

In [ ]:
# 定义一个简单的MJCF模型
xml = """
<mujoco>
  <worldbody>
    <!-- 定义一个红色的盒子，位于原点 -->
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <!-- 定义一个绿色的球体，偏移到(0.2, 0.2, 0.2)位置 -->
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# 从XML字符串加载模型
model = mujoco.MjModel.from_xml_string(xml)

上面的 `xml` 字符串使用MuJoCo的[MJCF](http://www.mujoco.org/book/modeling.html)（MuJoCo建模格式）编写，这是一种基于[XML](https://en.wikipedia.org/wiki/XML#Key_terminology)的建模语言。

**关键概念解析：**
- `<mujoco>` - 根元素，所有MuJoCo模型都必须包含此元素。最简单的有效模型就是 `<mujoco/>`
- `<worldbody>` - 世界主体，代表全局坐标系的原点，所有物理元素都必须位于其中
- `<geom>` - 几何体元素，定义可视化和碰撞检测的形状

**重要特性：**
- **默认值机制**：MJCF中的属性都有默认值。例如：
  - `red_box` 没有指定位置 → 默认位置是 `[0, 0, 0]`
  - `green_sphere` 没有指定类型 → 默认类型是 `sphere`（球体）
  
💡 **提示**：理解默认值机制对于编写简洁的MJCF模型非常重要。完整的默认值列表请参考[XML参考文档](https://mujoco.readthedocs.io/en/latest/XMLreference.html)。

## 理解mjModel：静态模型描述

`mjModel` 是MuJoCo的核心数据结构之一，包含所有**不随时间变化**的量。可以将其理解为物理世界的"蓝图"或"设计图"。

**mjModel包含的主要信息：**
- 📐 **几何信息**：几何体的形状、尺寸、颜色等
- 🔗 **拓扑结构**：刚体之间的连接关系
- ⚙️ **物理参数**：重力、时间步长、求解器设置等
- 🎨 **视觉属性**：材质、纹理、光照等

完整的字段描述请参考[`mjmodel.h`](https://github.com/google-deepmind/mujoco/blob/main/include/mujoco/mjmodel.h)头文件。

让我们查看模型中的一些基本信息：

In [ ]:
# 查看模型中的几何体数量
model.ngeom

In [ ]:
# 查看每个几何体的RGBA颜色值
# 第一行是红色盒子：[1., 0., 0., 1.] = 红色不透明
# 第二行是绿色球体：[0., 1., 0., 1.] = 绿色不透明
model.geom_rgba

## 便捷的命名访问机制

MuJoCo Python绑定提供了强大的[命名访问器](https://mujoco.readthedocs.io/en/latest/python.html#named-access)功能，让我们能够通过名称而非索引来访问模型元素。

**使用技巧：**
- 不带参数调用访问器会显示所有可用名称
- 这对于调试和探索模型结构非常有用

In [ ]:
# 尝试不带参数调用geom访问器，会显示所有可用的几何体名称
try:
  model.geom()
except KeyError as e:
  print(e)

不带属性名调用命名访问器会显示该对象的所有可用属性：

In [ ]:
# 访问green_sphere几何体的所有属性
# 这会显示该几何体的完整属性列表
model.geom('green_sphere')

让我们读取绿色球体的RGBA颜色值：

In [ ]:
# 直接访问green_sphere的rgba颜色值
model.geom('green_sphere').rgba

命名访问器本质上是对MuJoCo C API中[`mj_name2id`](https://mujoco.readthedocs.io/en/latest/APIreference.html?highlight=mj_name2id#mj-name2id)函数的Python封装，提供了更便捷的使用方式：

In [ ]:
# 使用底层C API函数mj_name2id来获取几何体ID
# 然后通过ID访问颜色数组
id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, "green_sphere")  # type: ignore
model.geom_rgba[id, :]

命名访问器还提供了只读的 `id` 和 `name` 属性，方便在ID和名称之间转换：

In [ ]:
# 演示ID和名称之间的转换
print('green_sphere的ID: ', model.geom('green_sphere').id)
print('ID为1的几何体名称: ', model.geom(1).name)
print('ID为0的刚体名称: ', model.body(0).name)  # 永远是'world'
# print("ID为0的刚体名称: ", model.body(1).name)  # 永远是'world'

💡 **注意**：第0个刚体始终是 `world`（世界主体），它代表全局坐标系，不能被重命名或删除。

`id` 和 `name` 属性在Python列表推导式中特别有用：

In [ ]:
# 使用列表推导式获取所有几何体的名称
[model.geom(i).name for i in range(model.ngeom)]

## 理解mjData：动态仿真状态

`mjData` 包含所有**随时间变化**的量，是仿真的"运行时状态"。

**mjData的核心组成：**
- ⏰ **仿真时间**：`data.time`
- 📍 **广义坐标**：
  - 位置：`data.qpos`（关节空间的位置）
  - 速度：`data.qvel`（关节空间的速度）
- 🌍 **笛卡尔坐标**：
  - 位置：`data.xpos`（世界坐标系中的位置）
  - 姿态：`data.xquat`（四元数表示的姿态）

要创建 `mjData`，只需要对应的 `mjModel`：

In [ ]:
# 创建mjData实例，用于存储仿真状态
data = mujoco.MjData(model)
for i in range(10):
    mujoco.mj_step(model, data)
    print(data.time)
    time.sleep(1)

`mjData` 不仅包含仿真状态，还包含许多**派生量**（从状态计算得出的值）。例如，物体在世界坐标系中的位置存储在 `data.geom_xpos` 中：

In [ ]:
# 打印几何体的世界坐标位置
# 注意：此时都是[0, 0, 0]，因为还没有进行运动学计算
print(data.geom_xpos)

🤔 **问题**：为什么两个几何体都在原点？我们不是将绿色球体偏移了吗？

**答案**：`mjData` 中的派生量需要**显式传播**。在我们的例子中，需要调用[`mj_kinematics`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-kinematics)函数来计算所有物体的全局笛卡尔位姿（不包括相机和光源）。

这是MuJoCo采用**延迟计算**策略的体现——只在需要时才计算派生量，以提高效率。

In [ ]:
# 调用mj_kinematics计算运动学，更新物体位置
mujoco.mj_kinematics(model, data)

# 现在可以看到正确的位置了
print('直接访问数组:\n', data.geom_xpos)

# MjData也支持命名访问
print('\n命名访问:\n', data.geom('green_sphere').xpos)

# 第二部分：渲染、仿真与动画

掌握了基础概念后，让我们学习如何渲染场景并创建动画。我们将重新加载模型以保持章节独立。

In [ ]:
# 重新定义模型（保持章节独立性）
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# 创建模型和数据
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# 创建渲染器并渲染场景
with mujoco.Renderer(model) as renderer:
  # 渲染并显示图像
  media.show_image(renderer.render())

🤔 **问题**：为什么渲染出来是黑色的？

**答案**：和之前一样，我们需要先传播 `mjData` 中的值。这次我们使用[`mj_forward`](https://mujoco.readthedocs.io/en/latest/APIreference/APIfunctions.html#mj-forward)函数，它会执行完整的前向动力学计算管道：

**mj_forward执行的操作：**
1. 计算运动学（位置、速度）
2. 计算动力学（力、力矩）
3. 更新传感器读数
4. 计算加速度

虽然对于静态渲染来说这些计算有些"过度"，但使用 `mj_forward` 是一个好习惯，可以确保所有需要的量都被正确计算。

另外，我们还需要更新渲染器持有的 `mjvScene`（视觉场景描述）：

In [ ]:
with mujoco.Renderer(model) as renderer:
  # 执行前向动力学计算，更新所有派生量
  mujoco.mj_forward(model, data)
  # 更新渲染场景
  renderer.update_scene(data)
  # 渲染并显示
  media.show_image(renderer.render())

成功了！但是图像有点暗。让我们添加一个光源来改善照明：

In [ ]:
# 添加光源改善照明
xml = """
<mujoco>
  <worldbody>
    <!-- 在(0, 0, 1)位置添加一个光源 -->
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
# 重新创建模型和数据
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# 渲染带光照的场景
with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)
  media.show_image(renderer.render())

好多了！

💡 **小贴士**：`mjModel` 实例中的所有值都是可写的。虽然通常建议在XML中修改值（更安全），但某些属性（如颜色）可以安全地在运行时修改：

In [ ]:
# 多次运行此单元格会看到不同的随机颜色
# 修改红色盒子的RGB值（保持Alpha=1）
model.geom('red_box').rgba[:3] = np.random.rand(3)

# 渲染更新后的颜色
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data)
  media.show_image(renderer.render())

# 创建物理仿真

现在让我们创建动态仿真。我们将使用MuJoCo的核心函数 `mj_step`，它推进仿真一个时间步长：$x_{t+h} = f(x_t)$

**重要概念：**
- **时间步长**：默认是2毫秒（0.002秒）
- **渲染频率**：我们选择60 FPS，而不是仿真的500 Hz
- **解耦设计**：物理仿真和视觉渲染是独立的，可以按不同频率运行

In [ ]:
# 设置仿真参数
duration = 3.8  # 仿真时长（秒）
framerate = 60  # 视频帧率（Hz）

# 仿真并生成视频
frames = []  # 存储视频帧
mujoco.mj_resetData(model, data)  # 重置状态和时间

with mujoco.Renderer(model) as renderer:
  
  while data.time < duration:
    # 推进一个时间步（默认2ms）
    mujoco.mj_step(model, data)
    t = data.time
    # 按照目标帧率采样图像
    # 注意：仿真频率(500Hz) != 渲染频率(60Hz)
    if len(frames) < t * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)

# 显示视频
media.show_video(frames, fps=framerate)

🤔 **问题**：视频在播放，但什么都没有移动，为什么？

**答案**：这个模型没有[自由度（DOF）](https://en.wikipedia.org/wiki/Degrees_of_freedom_(mechanics))。在MuJoCo中：
- 能够运动的物体被称为**刚体**（body）
- 刚体需要通过**关节**（joint）来获得运动能力
- 关节定义了刚体相对于其父体的运动方式

让我们创建一个新模型，添加刚体和铰链关节，并使用 `MjvOption` 可视化关节轴：

In [ ]:
# 创建带有运动自由度的模型
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <!-- 定义一个可以旋转的刚体 -->
    <body name="box_and_sphere" euler="0 0 -30">  <!-- 初始旋转30度 -->
      <!-- 添加铰链关节，绕斜轴旋转 -->
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# 创建可视化选项，显示关节轴
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

# 仿真参数
duration = 3.8  # (秒)
framerate = 60  # (Hz)

# 仿真并录制视频
frames = []
mujoco.mj_resetData(model, data)

with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      # 使用scene_option参数来显示关节
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

**关键观察点：**
- 使用 `euler="0 0 -30"` 将整个刚体绕Z轴（垂直轴）旋转了30度
- 这演示了**运动学树**的概念：元素的姿态总是相对于其**父体**定义
- 两个几何体都随着刚体一起旋转

物理参数存储在 `mjModel.opt` 中，例如时间步长：

In [ ]:
# 查看默认时间步长（2毫秒）
model.opt.timestep

让我们创造一个有趣的效果——反转重力方向：

In [ ]:
# 打印并修改重力设置
print('默认重力', model.opt.gravity)
# 反转重力方向（向上）
model.opt.gravity = (0, 0, 10)
print('反转后的重力', model.opt.gravity)

# 仿真并录制视频
frames = []
mujoco.mj_resetData(model, data)

with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

# 显示反重力效果
media.show_video(frames, fps=60)

我们也可以在XML中通过顶级 `<option>` 元素设置这些参数：
```xml
<mujoco>
  <option gravity="0 0 10"/>
  ...
</mujoco>
```

### 深入理解自由度（DOF）

**两种坐标表示方法的对比：**

1. **笛卡尔坐标（减法表示）**：
   - 每个刚体有6个自由度（3个平移 + 3个旋转）
   - 某些物理引擎使用这种方式
   - 缺点：效率低，需要额外处理约束

2. **广义坐标（加法表示）**- MuJoCo的选择：
   - 刚体默认没有自由度
   - 通过关节显式添加自由度
   - 优点：高效、稳定、自动满足约束

我们的模型有一个铰链关节，因此整个系统的状态由这个关节的角度和角速度完全定义：

In [ ]:
# 显示模型的自由度信息
print('模型总自由度数:', model.nv)
print('广义位置（关节角度）:', data.qpos)
print('广义速度（角速度）:', data.qvel)

这就是为什么在渲染前需要调用诸如[`mj_forward`](https://mujoco.readthedocs.io/en/latest/APIreference.html#mj-forward)之类的函数——笛卡尔位置是从广义坐标**派生**出来的，需要显式计算。

# 第三部分：高级示例 - 翻转陀螺仿真

让我们通过一个经典的物理现象来深入理解MuJoCo的能力。**翻转陀螺**（Tippe Top）是一个会自己翻转的旋转玩具（[视频演示](https://www.youtube.com/watch?v=kbYpVrdcszQ)，[维基百科](https://en.wikipedia.org/wiki/Tippe_top)）。

这个例子将展示：
- 🔄 自由关节（6自由度）的使用
- 🎯 关键帧的定义和应用
- 📊 仿真数据的记录和分析
- 🔬 复杂物理现象的准确模拟

In [ ]:
# 定义翻转陀螺模型
tippe_top = """
<mujoco model="tippe top">
  <!-- 使用4阶龙格-库塔积分器，提高精度 -->
  <option integrator="RK4"/>

  <!-- 定义纹理和材质 -->
  <asset>
    <!-- 棋盘格纹理 -->
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" texuniform="true" reflectance=".2"/>
  </asset>

  <!-- 设置平均尺寸 -->
  <statistic meansize=".01"/>

  <!-- 视觉设置 -->
  <visual>
    <global offheight="2160" offwidth="3840"/>
    <quality offsamples="8"/>
  </visual>

  <!-- 默认几何体属性 -->
  <default>
    <geom type="box" solref=".005 1"/>
    <default class="static">
      <geom rgba=".3 .5 .7 1"/>
    </default>
  </default>

  <!-- 更小的时间步长，提高稳定性 -->
  <option timestep="5e-4"/>

  <worldbody>
    <!-- 地面平面 -->
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <!-- 光源 -->
    <light pos="0 0 .6"/>
    <!-- 相机视角 -->
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    
    <!-- 陀螺主体 -->
    <body name="top" pos="0 0 .02">
      <!-- 6自由度自由关节 -->
      <freejoint/>
      <!-- 球形主体 -->
      <geom name="ball" type="sphere" size=".02" />
      <!-- 顶部柄 -->
      <geom name="stem" type="cylinder" pos="0 0 .02" size="0.004 .008"/>
      <!-- 底部配重（不可见，降低重心） -->
      <geom name="ballast" type="box" size=".023 .023 0.005"  pos="0 0 -.015"
       contype="0" conaffinity="0" group="3"/>
    </body>
  </worldbody>

  <!-- 关键帧：定义初始旋转状态 -->
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 1 0 0 0" qvel="0 0 0 0 1 200" />
  </keyframe>
</mujoco>
"""
# 加载模型
model = mujoco.MjModel.from_xml_string(tippe_top)
data = mujoco.MjData(model)

# 渲染初始状态
mujoco.mj_forward(model, data)
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data, camera="closeup")
  media.show_image(renderer.render())

**模型设计要点：**

1. **自由关节**：`<freejoint/>` 赋予陀螺完整的6自由度运动能力
2. **龙格-库塔积分器**：比默认的欧拉积分器精度更高，适合需要高精度的旋转动力学
3. **材质和纹理**：定义了棋盘格地面，增加视觉真实感
4. **隐形配重**：`ballast` 几何体降低重心，这是翻转现象发生的关键
5. **关键帧**：保存了初始旋转状态，便于重复实验
6. **相机定义**：预设了近景相机视角

让我们检查状态变量：

In [ ]:
# 查看自由关节的状态
# 位置：[x, y, z, qw, qx, qy, qz] - 3个平移 + 4个四元数
print('位置', data.qpos)
# 速度：[vx, vy, vz, ωx, ωy, ωz] - 3个线速度 + 3个角速度
print('速度', data.qvel)

**状态变量解析：**
- **位置（7个值）**：前3个是XYZ坐标，后4个是四元数表示的姿态
- **速度（6个值）**：前3个是线速度，后3个是角速度

💡 **为什么位置是7个值而速度是6个值？**
- 3D旋转需要4个参数（四元数）来完全描述，避免了欧拉角的万向锁问题
- 但角速度只需要3个参数（绕XYZ轴的旋转速度）

让我们运行仿真并观察翻转现象：

In [ ]:
# 仿真参数
duration = 7    # 仿真时长（秒）
framerate = 60  # 帧率（Hz）

# 仿真并录制翻转过程
frames = []
# 重置到关键帧0（初始旋转状态）
mujoco.mj_resetDataKeyframe(model, data, 0)

with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, "closeup")
      pixels = renderer.render()
      frames.append(pixels)

# 显示翻转陀螺视频
media.show_video(frames, fps=framerate)

### 数据记录与分析

`mjData` 结构包含了仿真过程中的所有动态变量和中间结果。让我们记录并分析陀螺的运动数据：

In [ ]:
# 初始化数据记录数组
timevals = []        # 时间序列
angular_velocity = [] # 角速度
stem_height = []     # 顶部柄的高度

# 重置并仿真
mujoco.mj_resetDataKeyframe(model, data, 0)
while data.time < duration:
  mujoco.mj_step(model, data)
  # 记录数据
  timevals.append(data.time)
  # 记录角速度（qvel的后3个分量）
  angular_velocity.append(data.qvel[3:6].copy())
  # 记录柄几何体的z坐标
  stem_height.append(data.geom_xpos[2,2])

# 设置图形参数
dpi = 120
width = 600
height = 800
figsize = (width / dpi, height / dpi)

# 创建子图
_, ax = plt.subplots(2, 1, figsize=figsize, dpi=dpi, sharex=True)

# 绘制角速度
ax[0].plot(timevals, angular_velocity)
ax[0].set_title('angular velocity')
ax[0].set_ylabel('radians / second')

# 绘制柄高度
ax[1].plot(timevals, stem_height)
ax[1].set_xlabel('time (seconds)')
ax[1].set_ylabel('meters')
_ = ax[1].set_title('stem height')

# 第四部分：混沌系统仿真

接下来我们将仿真一个**混沌摆**系统，类似于[旧金山探索馆](https://www.exploratorium.edu/exhibits/chaotic-pendulum)的展品。这个例子将展示：
- 🌀 混沌动力学的特性
- ⚡ 仿真性能的优化
- 📈 初始条件敏感性的可视化

下面是一个混沌摆的模型，类似于[旧金山探索馆](https://www.exploratorium.edu/exhibits/chaotic-pendulum)中的这个。

In [ ]:
# 定义混沌摆模型
chaotic_pendulum = """
<mujoco>
  <!-- 选项设置：1毫秒时间步，启用能量计算，禁用接触 -->
  <option timestep=".001">
    <flag energy="enable" contact="disable"/>
  </option>

  <!-- 默认设置 -->
  <default>
    <!-- 默认关节类型：铰链，绕Y轴旋转 -->
    <joint type="hinge" axis="0 -1 0"/>
    <!-- 默认几何体：胶囊形，半径2cm -->
    <geom type="capsule" size=".02"/>
  </default>

  <worldbody>
    <!-- 光源位置 -->
    <light pos="0 -.4 1"/>
    <!-- 固定相机视角 -->
    <camera name="fixed" pos="0 -1 0" xyaxes="1 0 0 0 0 1"/>
    
    <!-- 主摆体 -->
    <body name="0" pos="0 0 .2">
      <!-- 根关节 -->
      <joint name="root"/>
      <!-- 黄色横杆 -->
      <geom fromto="-.2 0 0 .2 0 0" rgba="1 1 0 1"/>
      <!-- 黄色竖杆 -->
      <geom fromto="0 0 0 0 0 -.25" rgba="1 1 0 1"/>
      
      <!-- 左侧摆 -->
      <body name="1" pos="-.2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="1 0 0 1"/>
      </body>
      
      <!-- 右侧摆 -->
      <body name="2" pos=".2 0 0">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 1 0 1"/>
      </body>
      
      <!-- 底部摆 -->
      <body name="3" pos="0 0 -.25">
        <joint/>
        <geom fromto="0 0 0 0 0 -.2" rgba="0 0 1 1"/>
      </body>
    </body>
  </worldbody>
</mujoco>
"""
# 加载模型
model = mujoco.MjModel.from_xml_string(chaotic_pendulum)
data = mujoco.MjData(model)
height = 480
width = 640

# 渲染初始状态
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, camera="fixed")
  media.show_image(renderer.render())

## 性能计时分析

让我们运行仿真并分析各个组件的性能：

In [ ]:
# 设置仿真参数
n_seconds = 6     # 仿真时长
framerate = 30    # 视频帧率
n_frames = int(n_seconds * framerate)  # 总帧数
frames = []
height = 240
width = 320

# 设置初始状态
mujoco.mj_resetData(model, data)
data.joint('root').qvel = 10  # 设置根关节初始角速度

# 仿真并记录帧，同时计时
frame = 0
sim_time = 0      # 仿真耗时
render_time = 0   # 渲染耗时
n_steps = 0       # 仿真步数

with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    # 仿真到下一个渲染时刻
    while data.time * framerate < i:
      tic = time.time()
      mujoco.mj_step(model, data)
      sim_time += time.time() - tic
      n_steps += 1
    
    # 渲染当前帧
    tic = time.time()
    renderer.update_scene(data, "fixed")
    frame = renderer.render()
    render_time += time.time() - tic
    frames.append(frame)

# 打印性能统计
step_time = 1e6*sim_time/n_steps  # 每步耗时（微秒）
step_fps = n_steps/sim_time       # 仿真频率（Hz）
print(f'仿真性能: {step_time:5.3g} μs/步  ({step_fps:5.0f}Hz)')

frame_time = 1e6*render_time/n_frames  # 每帧耗时（微秒）
frame_fps = n_frames/render_time       # 渲染频率（Hz）
print(f'渲染性能: {frame_time:5.3g} μs/帧 ({frame_fps:5.0f}Hz)')
print('\n')

# 显示视频
media.show_video(frames, fps=framerate)

**性能分析要点：**
- 物理仿真速度极快（通常 >100,000 Hz）
- 渲染是主要的性能瓶颈
- 仿真和渲染是解耦的，可以独立优化

## 混沌特性演示

混沌系统的标志是**初始条件的微小差异会导致完全不同的长期行为**：

In [ ]:
# 混沌系统演示：微小扰动导致巨大差异
PERTURBATION = 1e-7  # 初始条件扰动幅度
SIM_DURATION = 10    # 仿真时长（秒）
NUM_REPEATS = 8      # 重复次数

# 预分配数组
n_steps = int(SIM_DURATION / model.opt.timestep)
sim_time = np.zeros(n_steps)
angle = np.zeros(n_steps)
energy = np.zeros(n_steps)

# 准备绘图
_, ax = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

# 用微小不同的初始条件仿真多次
for _ in range(NUM_REPEATS):
  # 初始化
  mujoco.mj_resetData(model, data)
  data.qvel[0] = 10  # 根关节速度
  # 添加微小随机扰动
  data.qvel[:] += PERTURBATION * np.random.randn(model.nv)

  # 仿真
  for i in range(n_steps):
    mujoco.mj_step(model, data)
    sim_time[i] = data.time
    angle[i] = data.joint('root').qpos
    # 记录总能量（动能 + 势能）
    energy[i] = data.energy[0] + data.energy[1]

  # 绘图
  ax[0].plot(sim_time, angle)
  ax[1].plot(sim_time, energy)

# 完成绘图
ax[0].set_title('root angle')
ax[0].set_ylabel('radian')
ax[1].set_title('total energy')
ax[1].set_ylabel('Joule')
ax[1].set_xlabel('second')
plt.tight_layout()

## 时间步长与数值精度

**问题**：为什么能量会变化？在没有摩擦或阻尼的情况下，系统应该守恒能量。

**答案**：这是由于时间离散化导致的数值误差。

让我们研究时间步长对能量守恒的影响：

In [ ]:
# 研究时间步长对能量守恒的影响
SIM_DURATION = 10  # 仿真时长（秒）
TIMESTEPS = np.power(10, np.linspace(-2, -4, 5))  # 10ms到0.1ms的时间步长

# 准备绘图
_, ax = plt.subplots(1, 1)

for dt in TIMESTEPS:
  # 设置时间步长
  model.opt.timestep = dt

  # 分配数组
  n_steps = int(SIM_DURATION / model.opt.timestep)
  sim_time = np.zeros(n_steps)
  energy = np.zeros(n_steps)

  # 初始化
  mujoco.mj_resetData(model, data)
  data.qvel[0] = 9  # 根关节速度

  # 仿真
  print('{} 步，时间步长 = {:2.2g}ms'.format(n_steps, 1000*dt))
  for i in range(n_steps):
    mujoco.mj_step(model, data)
    sim_time[i] = data.time
    energy[i] = data.energy[0] + data.energy[1]

  # 绘制能量曲线
  ax.plot(sim_time, energy, label='timestep = {:2.2g}ms'.format(1000*dt))

# 完成绘图
ax.set_title('Energy')
ax.set_ylabel('Joule')
ax.set_xlabel('second')
ax.legend(frameon=True)
plt.tight_layout()

## 时间步长与仿真稳定性

当时间步长过大时，仿真会变得不稳定甚至发散：

In [ ]:
# 演示过大时间步长导致的发散
SIM_DURATION = 10  # 仿真时长（秒）
TIMESTEPS = np.power(10, np.linspace(-2, -1.5, 7))  # 10ms到31.6ms

# 获取绘图轴
ax = plt.gca()

for dt in TIMESTEPS:
    # 设置时间步长
    model.opt.timestep = dt

    # 分配数组（用NaN初始化以处理提前终止）
    n_steps = int(SIM_DURATION / model.opt.timestep)
    sim_time = np.zeros(n_steps) * np.nan
    energy = np.zeros(n_steps) * np.nan
    speed = np.zeros(n_steps) * np.nan

    # 初始化
    mujoco.mj_resetData(model, data)
    data.qvel[0] = 11  # 设置根关节速度

    # 仿真
    print("仿真 {} 步，时间步长 = {:2.2g}ms".format(n_steps, 1000 * dt))
    for i in range(n_steps):
        mujoco.mj_step(model, data)
        # 检查是否出现警告（发散）
        if data.warning.number.any():
            warning_index = np.nonzero(data.warning.number)[0][0]
            warning = mujoco.mjtWarning(warning_index).name
            print(f"由于发散（{warning}）在第 {i} 步停止。\n")
            break
        sim_time[i] = data.time
        # 记录速度绝对值之和作为能量代理
        energy[i] = sum(abs(data.qvel))  # type: ignore
        speed[i] = np.linalg.norm(data.qvel)

    # 绘图
    ax.plot(sim_time, energy, label='timestep = {:2.2g}ms'.format(1000*dt))
    ax.set_yscale('log')  # 使用对数坐标

# 完成绘图
ax.set_ybound(1, 1e3)
ax.set_title('Energy')
ax.set_ylabel('Joule')
ax.set_xlabel('second')
ax.legend(frameon=True, loc='lower right')
plt.tight_layout()

# 第五部分：接触动力学

接触和碰撞是机器人仿真的核心。让我们深入研究MuJoCo的接触处理机制：

In [ ]:
# 定义带接触的自由体模型
free_body_MJCF = """
<mujoco>
  <!-- 纹理和材质定义 -->
  <asset>
    <!-- 棋盘格纹理 -->
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
    rgb2=".2 .3 .4" width="300" height="300" mark="edge" markrgb=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true"
    reflectance=".2"/>
  </asset>

  <worldbody>
    <!-- 跟踪质心的光源 -->
    <light pos="0 0 1" mode="trackcom"/>
    <!-- 地面，带接触参数 -->
    <geom name="ground" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid" 
          solimp=".99 .99 .01" solref=".001 1"/>
    
    <!-- 自由运动的盒子和球体 -->
    <body name="box_and_sphere" pos="0 0 0">
      <freejoint/>  <!-- 6自由度 -->
      <!-- 红色盒子，带接触参数 -->
      <geom name="red_box" type="box" size=".1 .1 .1" rgba="1 0 0 1" 
            solimp=".99 .99 .01" solref=".001 1"/>
      <!-- 绿色球体 -->
      <geom name="green_sphere" size=".06" pos=".1 .1 .1" rgba="0 1 0 1"/>
      <!-- 固定相机视角 -->
      <camera name="fixed" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2"/>
      <!-- 跟踪相机视角 -->
      <camera name="track" pos="0 -.6 .3" xyaxes="1 0 0 0 1 2" mode="track"/>
    </body>
  </worldbody>
</mujoco>
"""
# 加载模型
model = mujoco.MjModel.from_xml_string(free_body_MJCF)
data = mujoco.MjData(model)
height = 400
width = 600

# 渲染初始状态
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, "fixed")
  media.show_image(renderer.render())

让我们创建一个慢动作动画，可视化接触点和接触力：

In [ ]:
# 慢动作渲染，可视化接触点和接触力
n_frames = 200
height = 240
width = 320
frames = []

# 创建可视化选项
options = mujoco.MjvOption()
mujoco.mjv_defaultOption(options)
# 启用接触点可视化
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
# 启用接触力可视化
options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True
# 启用透明模式
options.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True

# 调整接触可视化元素的比例
model.vis.scale.contactwidth = 0.1   # 接触点宽度
model.vis.scale.contactheight = 0.03 # 接触点高度
model.vis.scale.forcewidth = 0.05    # 力矢量宽度
model.vis.map.force = 0.3             # 力矢量缩放

# 设置随机初始旋转速度
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 5*np.random.randn(3)  # 随机角速度

# 仿真并录制视频
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    # 1/4倍速仿真
    while data.time < i/120.0:
      mujoco.mj_step(model, data)
    # 使用跟踪相机渲染
    renderer.update_scene(data, "track", options)
    frame = renderer.render()
    frames.append(frame)

# 显示慢动作视频
media.show_video(frames, fps=30)

## 接触力的详细分析

让我们重新运行仿真并深入分析接触相关的物理量：

In [ ]:
# 详细分析接触力
n_steps = 499

# 分配数组
sim_time = np.zeros(n_steps)
ncon = np.zeros(n_steps)          # 接触数量
force = np.zeros((n_steps,3))     # 接触力
velocity = np.zeros((n_steps, model.nv))      # 速度
penetration = np.zeros(n_steps)   # 穿透深度
acceleration = np.zeros((n_steps, model.nv))  # 加速度
forcetorque = np.zeros(6)         # 力和力矩

# 设置随机初始旋转速度
mujoco.mj_resetData(model, data)
data.qvel[3:6] = 2*np.random.randn(3)

# 仿真并记录数据
for i in range(n_steps):
    mujoco.mj_step(model, data)
    sim_time[i] = data.time
    ncon[i] = data.ncon  # 活动接触数
    velocity[i] = data.qvel[:]
    acceleration[i] = data.qacc[:]

    # 遍历所有活动接触
    for j, c in enumerate(data.contact):  # type: ignore
        # 计算接触力
        mujoco.mj_contactForce(model, data, j, forcetorque)  # type: ignore
        force[i] += forcetorque[0:3]  # 累加所有接触力
        penetration[i] = min(penetration[i], c.dist)  # 记录最大穿透

    # 另一种获取接触力的方法：
    # force[i] += data.qfrc_constraint[0:3]

# 创建子图
_, ax = plt.subplots(3, 2, sharex=True, figsize=(10, 10))

# 接触力
lines = ax[0,0].plot(sim_time, force)
ax[0,0].set_title('Contact Forces')
ax[0,0].set_ylabel('Newton')
ax[0,0].legend(lines, ('Normal z', 'Friction x', 'Friction y'))

# 加速度
ax[1,0].plot(sim_time, acceleration)
ax[1,0].set_title('Acceleration')
ax[1,0].set_ylabel('(m,rad)/s²')
ax[1,0].legend(['ax', 'ay', 'az', 'αx', 'αy', 'αz'])

# 速度
ax[2,0].plot(sim_time, velocity)
ax[2,0].set_title('Velocity')
ax[2,0].set_ylabel('(m,rad)/s')
ax[2,0].set_xlabel('second')
ax[2,0].legend(['vx', 'vy', 'vz', 'ωx', 'ωy', 'ωz'])

# 接触数量
ax[0,1].plot(sim_time, ncon)
ax[0,1].set_title('Contact Count')
ax[0,1].set_yticks(range(6))

# 法向力（对数坐标）
ax[1,1].plot(sim_time, force[:,0])
ax[1,1].set_yscale('log')
ax[1,1].set_title('Normal Force (z) - Log Scale')
ax[1,1].set_ylabel('Newton')
# 绘制mg参考线
z_gravity = -model.opt.gravity[2]
mg = model.body("box_and_sphere").mass[0] * z_gravity
mg_line = ax[1,1].plot(sim_time, np.ones(n_steps)*mg, label='m*g', linewidth=1)
ax[1,1].legend()

# 穿透深度
ax[2,1].plot(sim_time, 1000*penetration)
ax[2,1].set_title('Penetration Depth')
ax[2,1].set_ylabel('mm')
ax[2,1].set_xlabel('second')

plt.tight_layout()

## 摩擦力的影响

摩擦是接触动力学的重要组成部分。让我们观察不同摩擦系数的效果：

In [ ]:
# 演示不同摩擦系数的效果
MJCF = """
<mujoco>
  <asset>
    <!-- 棋盘格纹理 -->
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6"
     texuniform="true" reflectance=".2"/>
     <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>

  <!-- 默认设置 -->
  <default>
    <geom type="box" size=".05 .05 .05" />
    <joint type="free"/>
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <!-- 倾斜的地面，低摩擦 -->
    <geom name="ground" type="plane" size=".5 .5 10" material="grid"
     zaxis="-.3 0 1" friction=".1"/>
    <!-- 侧视相机 -->
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    
    <!-- 低摩擦盒子 -->
    <body pos="0 0 .1">
      <joint/>
      <geom/>
    </body>
    
    <!-- 高摩擦盒子 -->
    <body pos="0 .2 .1">
      <joint/>
      <geom friction=".33"/>
    </body>
  </worldbody>

</mujoco>
"""
n_frames = 60
height = 300
width = 300
frames = []

# 加载模型
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)

# 仿真并录制视频
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_resetData(model, data)
  for i in range(n_frames):
    while data.time < i/30.0:
      mujoco.mj_step(model, data)
    renderer.update_scene(data, "y")
    frame = renderer.render()
    frames.append(frame)

# 显示摩擦对比
media.show_video(frames, fps=30)

# 第六部分：腱系统、执行器和传感器

MuJoCo提供了丰富的机械系统建模能力。让我们探索腱系统、执行器和传感器的使用：

In [ ]:
# 演示腱系统、执行器和传感器
MJCF = """
<mujoco>
  <asset>
    <!-- 棋盘格纹理 -->
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="1 1"
     texuniform="true" reflectance=".2"/>
  </asset>

  <worldbody>
    <light name="light" pos="0 0 1"/>
    <!-- 地面 -->
    <geom name="floor" type="plane" pos="0 0 -.5" size="2 2 .1" material="grid"/>
    <!-- 锚点（腱的固定端） -->
    <site name="anchor" pos="0 0 .3" size=".01"/>
    <!-- 固定相机 -->
    <camera name="fixed" pos="0 -1.3 .5" xyaxes="1 0 0 0 1 2"/>

    <!-- 支柱 -->
    <geom name="pole" type="cylinder" fromto=".3 0 -.5 .3 0 -.1" size=".04"/>
    <!-- 球棒 -->
    <body name="bat" pos=".3 0 -.1">
      <!-- 铰链关节，带阻尼 -->
      <joint name="swing" type="hinge" damping="1" axis="0 0 1"/>
      <!-- 球棒几何体 -->
      <geom name="bat" type="capsule" fromto="0 0 .04 0 -.3 .04"
       size=".04" rgba="0 0 1 1"/>
    </body>

    <!-- 目标物体 -->
    <body name="box_and_sphere" pos="0 0 0">
      <joint name="free" type="free"/>
      <geom name="red_box" type="box" size=".1 .1 .1" rgba="1 0 0 1"/>
      <geom name="green_sphere"  size=".06" pos=".1 .1 .1" rgba="0 1 0 1"/>
      <!-- 挂钩点（腱的移动端） -->
      <site name="hook" pos="-.1 -.1 -.1" size=".01"/>
      <!-- IMU传感器位置 -->
      <site name="IMU"/>
    </body>
  </worldbody>

  <!-- 腱系统：连接锚点和挂钩 -->
  <tendon>
    <spatial name="wire" limited="true" range="0 0.35" width="0.003">
      <site site="anchor"/>
      <site site="hook"/>
    </spatial>
  </tendon>

  <!-- 执行器：驱动球棒 -->
  <actuator>
    <motor name="my_motor" joint="swing" gear="1"/>
  </actuator>

  <!-- 传感器：加速度计 -->
  <sensor>
    <accelerometer name="accelerometer" site="IMU"/>
  </sensor>
</mujoco>
"""
# 加载模型
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)
height = 480
width = 480

# 渲染初始状态
with mujoco.Renderer(model, height, width) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data, "fixed")
  media.show_image(renderer.render())

## 执行器控制与传感器数据

让我们运行仿真，驱动球棒击打目标物体：

In [ ]:
# 设置仿真参数
n_frames = 180
height = 240
width = 320
frames = []
fps = 60.0
times = []       # 时间序列
sensordata = []  # 传感器数据

# 设置恒定的执行器控制信号
mujoco.mj_resetData(model, data)
data.ctrl = 20  # 施加20牛米的力矩

# 仿真并录制视频
with mujoco.Renderer(model, height, width) as renderer:
  for i in range(n_frames):
    # 仿真到下一帧时间
    while data.time < i/fps:
      mujoco.mj_step(model, data)
      times.append(data.time)
      # 记录加速度计数据
      sensordata.append(data.sensor('accelerometer').data.copy())
    # 渲染帧
    renderer.update_scene(data, "fixed")
    frame = renderer.render()
    frames.append(frame)

# 显示球棒击打过程
media.show_video(frames, fps=fps)

让我们绘制加速度计传感器的测量数据：

In [ ]:
# 绘制加速度计数据
ax = plt.gca()

# 绘制三轴加速度
ax.plot(np.asarray(times), np.asarray(sensordata), 
        label=[f"axis {v}" for v in ['x', 'y', 'z']])

# 完成绘图
ax.set_title('Accelerometer Measurements')
ax.set_ylabel('m/s²')
ax.set_xlabel('second')
ax.legend(frameon=True, loc='lower right')
plt.tight_layout()

💡 **观察要点**：物体被球棒击中的时刻在加速度计数据中清晰可见，展示了MuJoCo精确的碰撞检测和力传递计算。

# 第七部分：高级渲染技术

本节将展示MuJoCo的高级渲染功能，包括透明度、深度渲染、分割渲染等。

In [ ]:
# 重新加载第一个模型
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

# 基本渲染
with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)
  media.show_image(renderer.render())

In [ ]:
# @title 启用透明度和框架可视化 {vertical-output: true}

# 设置可视化选项
scene_option.frame = (
    mujoco.mjtFrame.mjFRAME_GEOM
)  # 显示几何体框架# type: ignore
scene_option.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True  # 启用透明度

# 渲染
with mujoco.Renderer(model) as renderer:
  renderer.update_scene(data, scene_option=scene_option)
  frame = renderer.render()
  media.show_image(frame)

In [ ]:
#@title 深度渲染 {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  # 启用深度渲染模式
  renderer.enable_depth_rendering()

  # 更新场景
  renderer.update_scene(data)

  # 深度是浮点数组，单位为米
  depth = renderer.render()

  # 将最近的值移到原点
  depth -= depth.min()
  # 按近射线平均距离的2倍缩放
  depth /= 2*depth[depth <= 1].mean()
  # 缩放到[0, 255]
  pixels = 255*np.clip(depth, 0, 1)

  # 显示深度图
  media.show_image(pixels.astype(np.uint8))

In [ ]:
#@title 分割渲染 {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  # 禁用深度渲染
  renderer.disable_depth_rendering()

  # 启用分割渲染
  renderer.enable_segmentation_rendering()

  # 更新场景
  renderer.update_scene(data)

  # 渲染分割图
  seg = renderer.render()

  # 第一个通道包含对象ID
  # 第二个通道seg[:, :, 1]包含对象类型
  geom_ids = seg[:, :, 0]
  # 无穷大映射到-1
  geom_ids = geom_ids.astype(np.float64) + 1
  # 缩放到[0, 1]
  geom_ids = geom_ids / geom_ids.max()
  pixels = 255*geom_ids
  
  # 显示分割图
  media.show_image(pixels.astype(np.uint8))

## 相机矩阵

有关相机矩阵的描述，请参见维基百科上的[相机矩阵](https://en.wikipedia.org/wiki/Camera_matrix)文章。

In [ ]:
def compute_camera_matrix(renderer, data):
  """计算3x4相机矩阵
  
  相机矩阵用于将世界坐标投影到像素坐标
  """
  # 更新场景以确保相机参数正确
  renderer.update_scene(data)
  
  # 获取立体相机的平均位置和方向
  pos = np.mean([camera.pos for camera in renderer.scene.camera], axis=0)
  z = -np.mean([camera.forward for camera in renderer.scene.camera], axis=0)
  y = np.mean([camera.up for camera in renderer.scene.camera], axis=0)
  rot = np.vstack((np.cross(y, z), y, z))
  fov = model.vis.global_.fovy

  # 平移矩阵 (4x4)
  translation = np.eye(4)
  translation[0:3, 3] = -pos

  # 旋转矩阵 (4x4)
  rotation = np.eye(4)
  rotation[0:3, 0:3] = rot

  # 焦距变换矩阵 (3x4)
  focal_scaling = (1./np.tan(np.deg2rad(fov)/2)) * renderer.height / 2.0
  focal = np.diag([-focal_scaling, focal_scaling, 1.0, 0])[0:3, :]

  # 图像矩阵 (3x3)
  image = np.eye(3)
  image[0, 2] = (renderer.width - 1) / 2.0
  image[1, 2] = (renderer.height - 1) / 2.0
  
  # 组合所有变换
  return image @ focal @ rotation @ translation

In [ ]:
#@title 从世界坐标投影到相机坐标 {vertical-output: true}

with mujoco.Renderer(model) as renderer:
  renderer.disable_segmentation_rendering()
  # 更新场景
  renderer.update_scene(data)

  # 获取红色盒子的世界坐标
  box_pos = data.geom_xpos[model.geom('red_box').id]  # 位置
  box_mat = data.geom_xmat[model.geom('red_box').id].reshape(3, 3)  # 旋转矩阵
  box_size = model.geom_size[model.geom('red_box').id]  # 尺寸
  
  # 计算盒子8个角点的局部坐标
  offsets = np.array([-1, 1]) * box_size[:, None]
  xyz_local = np.stack(list(itertools.product(*offsets))).T
  # 转换到世界坐标
  xyz_global = box_pos[:, None] + box_mat @ xyz_local

  # 相机矩阵需要齐次坐标 [x, y, z, 1]
  corners_homogeneous = np.ones((4, xyz_global.shape[1]), dtype=float)
  corners_homogeneous[:3, :] = xyz_global

  # 获取相机矩阵
  m = compute_camera_matrix(renderer, data)

  # 将世界坐标投影到像素空间
  # 参见: https://en.wikipedia.org/wiki/3D_projection#Mathematical_formula
  xs, ys, s = m @ corners_homogeneous
  # x和y在像素坐标系中
  x = xs / s
  y = ys / s

  # 渲染相机视图并叠加投影的角点坐标
  pixels = renderer.render()
  fig, ax = plt.subplots(1, 1)
  ax.imshow(pixels)
  ax.plot(x, y, '+', c='w')  # 白色十字标记角点
  ax.set_axis_off()

## 修改场景

让我们向 `mjvScene` 添加一些任意几何体。

In [ ]:
def get_geom_speed(model, data, geom_name):
    """计算几何体的速度"""
    geom_vel = np.zeros(6)
    geom_type = mujoco.mjtObj.mjOBJ_GEOM
    geom_id = data.geom(geom_name).id
    # 获取几何体的速度（线速度和角速度）
    mujoco.mj_objectVelocity(model, data, geom_type, geom_id, geom_vel, 0)  # type: ignore
    return np.linalg.norm(geom_vel)

def add_visual_capsule(scene, point1, point2, radius, rgba):
    """向mjvScene添加一个可视化胶囊"""
    if scene.ngeom >= scene.maxgeom:
        return
    scene.ngeom += 1  # 增加几何体计数
    # 初始化新胶囊
    mujoco.mjv_initGeom(
        scene.geoms[scene.ngeom - 1],
        mujoco.mjtGeom.mjGEOM_CAPSULE,  # type: ignore
        np.zeros(3),  # type: ignore
        np.zeros(3),  # type: ignore
        np.zeros(9),  # type: ignore
        rgba.astype(np.float32),
    )
    # 使用mjv_connector设置胶囊的两个端点
    mujoco.mjv_connector(
        scene.geoms[scene.ngeom - 1],
        mujoco.mjtGeom.mjGEOM_CAPSULE,  # type: ignore
        radius,
        point1,
        point2,
    )

# 时间、位置和速度的轨迹
times = []
positions = []
speeds = []
offset = model.jnt_axis[0]/16  # 沿关节轴的偏移

def modify_scene(scn):
  """绘制位置轨迹，速度决定宽度和颜色"""
  if len(positions) > 1:
    for i in range(len(positions)-1):
      # 根据速度设置颜色
      rgba=np.array((np.clip(speeds[i]/10, 0, 1),
                     np.clip(1-speeds[i]/10, 0, 1),
                     .5, 1.))
      # 根据速度设置半径
      radius=.003*(1+speeds[i])
      # 计算轨迹点
      point1 = positions[i] + offset*times[i]
      point2 = positions[i+1] + offset*times[i+1]
      add_visual_capsule(scn, point1, point2, radius, rgba)

duration = 6    # 仿真时长（秒）
framerate = 30  # 帧率（Hz）

# 仿真并录制视频
frames = []

# 重置状态和时间
mujoco.mj_resetData(model, data)
mujoco.mj_forward(model, data)

with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    # 记录轨迹数据
    positions.append(data.geom_xpos[data.geom("green_sphere").id].copy())
    times.append(data.time)
    speeds.append(get_geom_speed(model, data, "green_sphere"))
    
    # 仿真步进
    mujoco.mj_step(model, data)
    
    # 按帧率录制
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      # 修改场景，添加轨迹
      modify_scene(renderer.scene)
      pixels = renderer.render()
      frames.append(pixels)

# 显示带轨迹的视频
media.show_video(frames, fps=framerate)

## 同一场景中的多个帧

有时人们希望多次绘制相同的几何体，例如当模型跟踪来自动作捕捉的状态时，最好将数据可视化在模型旁边。与 `mjv_updateScene`（由 `Renderer` 的 `update_scene` 方法调用）不同，它在每次调用时清除场景，`mjv_addGeoms` 将向现有场景添加视觉几何体：

In [ ]:
# 获取MuJoCo的标准人形模型
print('从GitHub获取MuJoCo人形机器人XML描述：')
# !git clone https://github.com/google-deepmind/mujoco
with open('humanoid.xml', 'r') as f:
  xml = f.read()

# 加载模型，创建两个MjData实例
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)   # 主仿真数据
data2 = mujoco.MjData(model)  # 用于"幽灵"显示

# 设置仿真参数
duration = 3       # 仿真时长（秒）
framerate = 60     # 帧率（Hz）
data.qpos[0:2] = [-.5, -.5]  # 初始x-y位置（米）
data.qvel[2] = 4   # 初始垂直速度（米/秒）
ctrl_phase = 2 * np.pi * np.random.rand(model.nu)  # 控制相位
ctrl_freq = 1     # 控制频率

# "幽灵"模型的视觉选项
vopt2 = mujoco.MjvOption()
vopt2.flags[mujoco.mjtVisFlag.mjVIS_TRANSPARENT] = True  # 透明显示
pert = mujoco.MjvPerturb()  # 空的MjvPerturb对象
# 只显示动态物体（人形机器人），不重绘静态物体（地面）
catmask = mujoco.mjtCatBit.mjCAT_DYNAMIC

# 仿真并渲染
frames = []
with mujoco.Renderer(model, 480, 640) as renderer:
    while data.time < duration:
        # 正弦控制信号
        data.ctrl = np.sin(ctrl_phase + 2 * np.pi * data.time * ctrl_freq)
        mujoco.mj_step(model, data)

        if len(frames) < data.time * framerate:
            # 渲染主人形机器人
            renderer.update_scene(data)

            # 复制qpos到data2，横向移动人形机器人
            data2.qpos = data.qpos
            data2.qpos[0] += 1.5  # x方向偏移
            data2.qpos[1] += 1  # y方向偏移
            mujoco.mj_forward(model, data2)

            # 使用mjv_addGeoms添加幽灵人形到场景
            mujoco.mjv_addGeoms(model, data2, vopt2, pert, catmask, renderer.scene)  # type: ignore

            # 渲染并添加帧
            pixels = renderer.render()
            frames.append(pixels)

# 以半速播放视频
media.show_video(frames, fps=framerate/2)

## 动态相机控制

相机控制是创建专业仿真视频的关键技术。以下示例展示了如何实现电影级的相机运动效果。

**技术要点：**
- 平滑的相机轨迹过渡
- 从环绕视角到跟踪视角的无缝切换
- 动态调整相机参数以获得最佳视角

In [ ]:
#@title 加载"多米诺骨牌"模型

# 定义多米诺骨牌仿真场景
dominos_xml = """
<mujoco model="tippe top">
  <!-- 纹理和材质 -->
  <asset>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="2 2" texuniform="true" reflectance=".2"/>
  </asset>

  <!-- 统计信息 -->
  <statistic meansize=".01"/>

  <!-- 视觉设置 -->
  <visual>
    <global offheight="2160" offwidth="3840"/>
    <quality offsamples="8"/>
  </visual>

  <!-- 默认设置 -->
  <default>
    <geom type="box" solref=".005 1"/>
    <default class="static">
      <geom rgba=".3 .5 .7 1"/>
    </default>
  </default>

  <!-- 小时间步长，提高精度 -->
  <option timestep="5e-4"/>

  <worldbody>
    <!-- 地面 -->
    <geom name="floor" type="plane" size="3 3 .01" pos="-0.025 -0.295  0" material="grid"/>
    <!-- 斜坡 -->
    <geom name="ramp" pos=".25 -.45 -.03" size=".04 .1 .07" euler="-30 0 0" class="static"/>
    <!-- 光源 -->
    <light pos=".3 -.3 .8" mode="trackcom" diffuse="1 1 1" specular=".3 .3 .3"/>
    <light pos="0 -.3 .4" mode="targetbodycom" target="box" diffuse=".8 .8 .8" specular=".3 .3 .3"/>
    <!-- 相机 -->
    <camera name="top" pos="-0.37 -0.78 0.49" xyaxes="0.78 -0.63 0 0.27 0.33 0.9"/>

    <!-- 球（开始推倒多米诺） -->
    <body name="ball" pos=".25 -.45 .1">
      <freejoint name="ball"/>
      <geom name="ball" type="sphere" size=".02" rgba=".65 .81 .55 1"/>
    </body>

    <!-- 多米诺骨牌（逐渐增大） -->
    <body pos=".26 -.3 .03" euler="0 0 -90.0">
      <freejoint/>
      <geom size=".0015 .015 .03" rgba="1 .5 .5 1"/>
    </body>

    <body pos=".26 -.27 .04" euler="0 0 -81.0">
      <freejoint/>
      <geom size=".002 .02 .04" rgba="1 1 .5 1"/>
    </body>

    <body pos=".24 -.21 .06" euler="0 0 -63.0">
      <freejoint/>
      <geom size=".003 .03 .06" rgba=".5 1 .5 1"/>
    </body>

    <body pos=".2 -.16 .08" euler="0 0 -45.0">
      <freejoint/>
      <geom size=".004 .04 .08" rgba=".5 1 1 1"/>
    </body>

    <body pos=".15 -.12 .1" euler="0 0 -27.0">
      <freejoint/>
      <geom size=".005 .05 .1" rgba=".5 .5 1 1"/>
    </body>

    <body pos=".09 -.1 .12" euler="0 0 -9.0">
      <freejoint/>
      <geom size=".006 .06 .12" rgba="1 .5 1 1"/>
    </body>

    <!-- 跷跷板装置 -->
    <body name="seasaw_wrapper" pos="-.23 -.1 0" euler="0 0 30">
      <geom size=".01 .01 .015" pos="0 .05 .015" class="static"/>
      <geom size=".01 .01 .015" pos="0 -.05 .015" class="static"/>
      <geom type="cylinder" size=".01 .0175" pos="-.09 0 .0175" class="static"/>
      <body name="seasaw" pos="0 0 .03">
        <joint axis="0 1 0"/>
        <geom type="cylinder" size=".005 .039" zaxis="0 1 0" rgba=".84 .15 .33 1"/>
        <geom size=".1 .02 .005" pos="0 0 .01" rgba=".84 .15 .33 1"/>
      </body>
    </body>

    <!-- 目标盒子 -->
    <body name="box" pos="-.3 -.14 .05501" euler="0 0 -30">
      <freejoint name="box"/>
      <geom name="box" size=".01 .01 .01" rgba=".0 .7 .79 1"/>
    </body>
  </worldbody>
</mujoco>
"""
# 加载模型
model = mujoco.MjModel.from_xml_string(dominos_xml)
data = mujoco.MjData(model)

In [ ]:
#@title 固定相机渲染

# 设置仿真参数
duration = 2.5  # 仿真时长（秒）
framerate = 60  # 帧率（Hz）
height = 1024
width = 1440

# 仿真并录制视频
frames = []
mujoco.mj_resetData(model, data)  # 重置状态

with mujoco.Renderer(model, height, width) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, camera='top')
      pixels = renderer.render()
      frames.append(pixels)

# 显示多米诺骨牌效果
media.show_video(frames, fps=framerate)

In [ ]:
#@title 动态相机渲染

# 设置参数
duration = 3  # 仿真时长（秒）
height = 1024
width = 1440

# 找到盒子被抛出的时间（速度>2cm/s）
throw_time = 0.0
mujoco.mj_resetData(model, data)
while data.time < duration and not throw_time:
  mujoco.mj_step(model, data)
  box_speed = np.linalg.norm(data.joint('box').qvel[:3])
  if box_speed > 0.02:
    throw_time = data.time
assert throw_time > 0

def mix(time, t0=0.0, width=1.0):
  """S型混合函数"""
  t = (time - t0) / width
  s = 1 / (1 + np.exp(-t))
  return 1 - s, s

def unit_cos(t):
  """单位余弦函数，从(0,0)到(1,1)"""
  return 0.5 - np.cos(np.pi*np.clip(t, 0, 1))/2

def orbit_motion(t):
  """返回环绕轨迹参数"""
  distance = 0.9
  azimuth = 140 + 100 * unit_cos(t)  # 方位角变化
  elevation = -30
  lookat = data.geom('floor').xpos.copy()
  return distance, azimuth, elevation, lookat

def track_motion():
  """返回跟踪盒子的轨迹参数"""
  distance = 0.08
  azimuth = 280
  elevation = -10
  lookat = data.geom('box').xpos.copy()
  return distance, azimuth, elevation, lookat

def cam_motion():
  """返回混合的相机轨迹（环绕→跟踪）"""
  # 盒子抛出前的环绕轨迹
  d0, a0, e0, l0 = orbit_motion(data.time / throw_time)
  # 盒子跟踪轨迹
  d1, a1, e1, l1 = track_motion()
  # 混合时间
  mix_time = 0.3
  w0, w1 = mix(data.time, throw_time, mix_time)
  # 加权混合
  return w0*d0+w1*d1, w0*a0+w1*a1, w0*e0+w1*e1, w0*l0+w1*l1

# 创建相机对象
cam = mujoco.MjvCamera()
mujoco.mjv_defaultCamera(cam)

# 仿真并录制视频
framerate = 60  # 帧率（Hz）
slowdown = 4    # 4倍慢动作
mujoco.mj_resetData(model, data)
frames = []

with mujoco.Renderer(model, height, width) as renderer:
  while data.time < duration:
    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate * slowdown:
      # 更新相机参数
      cam.distance, cam.azimuth, cam.elevation, cam.lookat = cam_motion()
      renderer.update_scene(data, cam)
      pixels = renderer.render()
      frames.append(pixels)

# 显示动态相机效果
media.show_video(frames, fps=framerate)

# 总结与展望

恭喜您完成了MuJoCo物理引擎的完整教程！通过本教程，您已经掌握了：

## 🎯 核心知识点回顾

### 1. **基础概念**
- ✅ MuJoCo的架构：mjModel（静态）vs mjData（动态）
- ✅ MJCF建模语言的使用
- ✅ 广义坐标系统的理解
- ✅ 仿真循环的实现

### 2. **物理仿真**
- ✅ 多体动力学仿真
- ✅ 接触力和摩擦力建模
- ✅ 数值积分和稳定性
- ✅ 混沌系统的特性

### 3. **高级功能**
- ✅ 传感器和执行器系统
- ✅ 腱系统的使用
- ✅ 多种渲染模式（RGB、深度、分割）
- ✅ 场景修改和可视化增强

### 4. **实践技能**
- ✅ 创建复杂的机器人模型
- ✅ 实现控制算法
- ✅ 录制高质量仿真视频
- ✅ 性能优化和调试

## 🚀 下一步学习建议

### 1. **深入特定领域**
- 🤖 **机器人控制**：实现更复杂的控制算法（LQR、MPC等）
- 🦾 **强化学习**：将MuJoCo与Gymnasium、Stable-Baselines3集成
- 🦿 **仿生机器人**：研究四足、双足机器人的运动控制
- 🏭 **工业应用**：机械臂路径规划和抓取仿真

### 2. **高级主题探索**
- 📊 **MJX（JAX加速）**：使用GPU加速大规模仿真
- 🔌 **硬件接口**：实现仿真到真实机器人的控制
- 🧩 **插件开发**：创建自定义传感器和执行器
- 🌐 **分布式仿真**：多机器人协作仿真

### 3. **项目实践**
- 创建自己的机器人模型
- 设计和验证控制算法
- 参与开源项目贡献
- 发表研究成果

## 📚 推荐资源

### 官方资源
- [MuJoCo文档](https://mujoco.readthedocs.io/)
- [MuJoCo论坛](https://github.com/google-deepmind/mujoco/discussions)
- [示例模型库](https://github.com/google-deepmind/mujoco/tree/main/model)

### 学习社区
- MuJoCo GitHub Issues：技术问题讨论
- 机器人学相关会议：ICRA、IROS、RSS
- 在线课程和教程

## 💡 最后的建议

1. **动手实践**：理论学习要与实践相结合
2. **循序渐进**：从简单模型开始，逐步增加复杂度
3. **关注细节**：物理参数的微小变化可能产生巨大影响
4. **保持好奇**：探索MuJoCo的各种可能性

## 🙏 致谢

感谢您完成本教程的学习！MuJoCo是一个强大而优雅的物理引擎，希望本教程能够帮助您在机器人仿真的道路上走得更远。

如果您有任何问题或建议，欢迎：
- 在GitHub上提出Issue
- 参与社区讨论
- 分享您的项目和经验

**祝您在机器人仿真的世界中探索愉快！** 🚀✨